# **Value-at-Risk (VaR) Estimator (Work In Progress)**

## **Contents**
1. [Introduction](#Introduction)
2. [Risk Functions](#Risk-Functions)
   * [Value-at-Risk (VaR)](#Value-at-Risk-(VaR))
   * [Expected Shortfall (ES)](#Expected-Shortfall-(ES))
   * [Robust Value-at-Risk (RVaR)](#Robust-Value-at-Risk-(RVaR))
3. [Historical Method](#Historical-Method)
   * [Application of the Historical Method](#Application-of-the-Historical-Method)
   * [Python Implementation of the Historical-Method](#Python-Implementation-of-the-Historical-Method)
4. [Parametric Method](#Parametric-Method)
   * [Application of the Parametric Method](#Application-of-the-Parametric-Method)
   * [Python Implementation of the Parametric Method](#Python-Implementation-of-the-Parametric-Method)
5. [Monte Carlo Method](#Monte-Carlo-Method)
   * [Application of the Parametric Method](#Application-of-the-Parametric-Method)
   * [Python Implementation of the Parametric Method](#Python-Implementation-of-the-Parametric-Method)
6. [Credits](#Credits)

## **Introduction**
The purpose of this project is to provide a method to immediately estimate, using Python, the Value-at-Risk (VaR), and related risk functions, of a portfolio of risk-bearing assets. The `time`, `datetime`, `numpy`, `scipy`, `sympy`, `pandas`, `matplotlib`, `seaborn`, `yfinance`, `pypfopt`, `riskfolio` and `warnings` libraries will be imported. Moreover, a random seed value is set for reprodicibility and the plot styles are modified for aesthetic reasons.

In [ ]:
import time as tm
import datetime as dt
import numpy as np
import scipy as sp
import sympy as smp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pypfopt as ppf
import riskfolio as rsk
import warnings as wrn

np.random.seed(30)
palette = sns.color_palette("viridis_r", as_cmap=True)
sns.set_style(style = "whitegrid", rc = {"font.family":"Times New Roman", "font.weight":"bold"})
wrn.simplefilter(action='ignore', category=FutureWarning)

tickers = ["MSFT", "AAPL", "NVDA", "AMZN"]
start_date = dt.datetime.strptime("2013-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2018-01-01", "%Y-%m-%d")
data = yf.download(tickers, start_date, end_date, progress = False)["Adj Close"]
data.head()

## **Risk Functions**
Let $\left(\Omega, \mathscr{F}, \mathbb{P}\right)$ be a probability space, let $(\mu, \sigma)$ be an ordered pair in $\mathbb{R}^{*} \times \mathbb{R}_{+}^{*}$, let $(S_{t})_{t \in \mathbb{R}_{+}}$ be a geometric Brownian motion with percentage drift $\mu$ and percentage volatility $\sigma$ on $\left(\Omega, \mathscr{F}, \mathbb{P}\right)$ such that $S_{0} \substack{\mathbb{P}\text{-a.s.} \\ \neq} 0$ and let $(h, \alpha)$ be an ordered pair in $\mathbb{R}_{+}^{*} \times ]0,1[$.
- The *profit-and-loss distribution of $(S_{t})_{t \in \mathbb{R}_{+}}$ at horizon $h$* is the stochastic process denoted by $(L_{t, h})_{t \in \mathbb{R}_{+}}$ and defined by:
$$\forall (\omega, t) \in \Omega \times \mathbb{R}_{+}, \enspace L_{t, h}(\omega) = S_{t + h}(\omega) - S_{t}(\omega)$$
- The *relative return distribution of $(S_{t})_{t \in \mathbb{R}_{+}}$ at horizon $h$* is the stochastic process denoted by $(R_{t, h})_{t \in \mathbb{R}_{+}}$ and defined by:
$$\forall (\omega, t) \in \Omega \times \mathbb{R}_{+}, \enspace R_{t, h}(\omega) = \frac{S_{t + h}(\omega) - S_{t}(\omega)}{S_{t}(\omega)}$$
Here, the stochastic process $(S_{t})_{t \in \mathbb{R}_{+}}$ can represent the price of a single risk-bearing financial asset, or the overall value of a portfolio of risk-bearning financial asset.

### **Value-at-Risk (VaR)**
For every positive real number $t$ in $\mathbb{R}_{+}$ the *Value-at-Risk (VaR) of $S_{t}$ at siginificance level $\alpha$ and horizon $h$* is the real number denoted by $\mathrm{VaR}_{\alpha, h}\left(S_{t}\right)$ and defined by:
$$\mathrm{VaR}_{\alpha, h}\left(S_{t}\right) = -\inf\left(\left\{x \in \mathbb{R} \colon \enspace F_{L_{t, h}}(x) > \alpha\right\}\right)$$

### **Expected Shortfall (ES)**
For every positive real number $t$ in $\mathbb{R}_{+}$ the *Expected Shortfall (ES) of $S_{t}$ at siginificance level $\alpha$ and horizon $h$* is the real number denoted by $\mathrm{ES}_{\alpha, h}\left(S_{t}\right)$ and defined by:
$$\mathrm{ES}_{\alpha, h}\left(S_{t}\right) = -\frac{1}{\alpha}\int_{0}^{\alpha}\mathrm{VaR}_{\gamma, h}\left(S_{t}\right)\mathrm{d}\gamma$$
The expected shortfall is also called the Conditional Value-at-Risk (CVaR).


### **Robust Value-at-Risk (RVaR)**
Let $\beta$ be a strictly positive real number in $]0,1[$ such that $\alpha < \beta$. For every positive real number $t$ in $\mathbb{R}_{+}$ the *Robust Value-at-Risk of $S_{t}$ between siginificance levels $\alpha$ and $\beta$ and at horizon $h$* is the real number denoted by $\mathrm{RVaR}_{\alpha, \beta, h}\left(S_{t}\right)$ and defined by:
$$\mathrm{RVaR}_{\alpha, \beta, h}\left(S_{t}\right) = -\frac{1}{\beta - \alpha}\int_{\alpha}^{\beta}\mathrm{VaR}_{\gamma, h}\left(S_{t}\right)\mathrm{d}\gamma$$

In most cases, the exact values of each of the above three risk functions cannot be calculated directly. There are, however, three methods that stand out in calculating each of them: 
- The historical method (which estimates the values of the risk functions from the quantiles of past data of the profit-and-loss distribution of the underlying geometric Brownian motion);
- The parametric method (which estimates the values of the risk functions from the parameters of the distribution of past data of the profit-and-loss distribution of the underlying geometric Brownian motion), and;
- The Monte Carlo method (which estimates the values of the risk functions by repeated random sampling from the distribution of past data of the profit-and-loss distribution of the underlying geometric Brownian motion)
The details of each of the above three methods, as well as their Python implementation, is given in the following sections.

## **Historical Method**
### **Application of the Historical Method**
The following is a general procedure for estimating the risk functions via the historical method:
1. Retrieve the historical price data of the underlying asset or portfolio of assets, the frequency being the time horizon of choice;
2. Calculate the profit-and-loss distribution corresponding to this historical price data;
3. Determine the relevant estimates of the values of the risk functions as follows:
   * In order to calculate $\mathrm{VaR}_{\alpha, h}(S_{t})$, select the negative of the $\alpha$-quantile of the profit-and-loss distribution.
   * In order to calculate $\mathrm{ES}_{\alpha, h}(S_{t})$, calculate the negative of the mean of all values in the profit-and-loss distribution not exceeding its $\alpha$-quantile.
   * In order to calculate $\mathrm{RVaR}_{\alpha, \beta, h}(S_{t})$, calculate the negative of the mean of all values in the profit-and-loss distribution not between its $\alpha$-quantile and its $\beta$-quantile.
The historical method makes no *a priori* assumption about the profit-and-loss distribution. It only retireves the quantiles of interest.

### **Python Implementation of the Historical Method**
The Python implementation of the historical method for estimating the values of the above three risk functions is given in the code cell below.

In [7]:
def Historical_VaR(tickers, start_date, end_date, weights, show_performance_metrics = False, show_visualizations = False):
    data = yf.download(tickers, start_date, end_date)["Adj Close"]


### TO BE CONTINUED

## **Parametric Method**
### **Application of the Parametric Method**
### **Python Implementation of the Parametric Method**

## **Monte Carlo Method**
### **Application of the Monte Carlo Method**
### **Python Implementation of the Monte Carlo Method**

## **Credits**
* Wikipedia contributors, "[Value at risk](https://en.wikipedia.org/w/index.php?title=Value_at_risk&oldid=1207006629)," Wikipedia, The Free Encyclopedia (accessed February 29, 2024).
* Wikipedia contributors, "[Expected shortfall](https://en.wikipedia.org/w/index.php?title=Expected_shortfall&oldid=1191488388)," Wikipedia, The Free Encyclopedia (accessed February 29, 2024).